# Functional Encryption - Classification and information leakage

 
We do the same operation than in Part 9 with a model which has been more trained to resist against adversaries

# 9 Converting the model to integers (Extended results)


Load torch and syft packages

In [23]:
# Allow to load packages from parent
import sys, os
sys.path.insert(1, os.path.realpath(os.path.pardir))

In [24]:
import torch

In [25]:
from math import log2, ceil

import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils

import learn
from learn import collateral

In [26]:
PRIVATE_OUTPUT_SIZE = 4
N_CHARS = 10
N_FONTS = 2

In [ ]:
fonts = ['cursive', 'Georgia']
letter = "6"

Let's load the quadratic model that we saved in Part 8! _Be sure that the path and file name match._

In [27]:
class CollateralNet(nn.Module):
    def __init__(self, private_output_size):
        super(CollateralNet, self).__init__()
        self.proj1 = nn.Linear(784, 40)
        self.diag1 = nn.Linear(40, private_output_size, bias=False)

        # --- FFN for characters
        self.lin1 = nn.Linear(private_output_size, 32)
        self.lin2 = nn.Linear(32, N_CHARS)

        # --- Junction
        self.jct = nn.Linear(private_output_size, 784)

        # --- CNN for families
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, N_FONTS)
        
        # FFN for families
        self.lin3 = nn.Linear(private_output_size, 64)
        self.lin4 = nn.Linear(64, 32)
        self.lin5 = nn.Linear(32, 16)
        self.lin6 = nn.Linear(16, 8)
        self.lin7 = nn.Linear(8, N_CHARS)

    def quad(self, x):
        # --- Quadratic
        x = x.view(-1, 784)
        x = self.proj1(x)
        x = x * x
        x = self.diag1(x)
        return x

    def char_net(self, x):
        # --- FFN
        x = F.relu(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

    def font_net(self, x):
        # --- Junction
        x = self.jct(x)
        x = x.view(-1, 1, 28, 28)

        # --- CNN
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def forward_char(self, x):
        x = self.quad(x)
        x = self.char_net(x)
        return F.log_softmax(x, dim=1)

    def forward_font(self, x):
        x = self.quad(x)
        x = self.font_net(x)
        return F.log_softmax(x, dim=1)
    
    # We add the ability to freeze some layers to ensure that the collateral task does
    # not modify the quadratic net
    
    def get_params(self, net):
        """Select the params for a given part of the net"""
        if net == 'quad':
            layers = [self.proj1, self.diag1]
        elif net == 'char':
            layers = [self.lin1, self.lin2]
        elif net == 'font':
            layers = [self.jct, self.fc1, self.fc2, self.conv1, self.conv2]
        else:
            raise AttributeError(f'{net} type not recognized')
        params = [p for layer in layers for p in layer.parameters()]
        return params

    def freeze(self, net):
        """Freeze a part of the net"""
        net_params = self.get_params(net)
        for param in net_params:
            param.requires_grad = False

    def unfreeze(self):
        """Unfreeze the net"""
        for param in self.parameters():
            param.requires_grad = True

In [36]:
#path = '../data/models/quadconvnet_part8.pt'
path = 'models/quadconvnet_0.5_0.002_1.7_4_0_par2.pt'
model = CollateralNet(private_output_size=PRIVATE_OUTPUT_SIZE)
results = {}

model.load_state_dict(torch.load(path))
model.eval()

CollateralNet(
  (proj1): Linear(in_features=784, out_features=40, bias=True)
  (diag1): Linear(in_features=40, out_features=4, bias=False)
  (lin1): Linear(in_features=4, out_features=32, bias=True)
  (lin2): Linear(in_features=32, out_features=10, bias=True)
  (jct): Linear(in_features=4, out_features=784, bias=True)
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (lin3): Linear(in_features=4, out_features=64, bias=True)
  (lin4): Linear(in_features=64, out_features=32, bias=True)
  (lin5): Linear(in_features=32, out_features=16, bias=True)
  (lin6): Linear(in_features=16, out_features=8, bias=True)
  (lin7): Linear(in_features=8, out_features=10, bias=True)
)

Here is a function to analyse a tensor distribution

In [37]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def print_hist(data):
    x = data.view(-1).abs()
    # the histogram of the data
    n, bins, patches = plt.hist(x, 50, density=True, facecolor='g', alpha=0.75)
    plt.xlabel('Weight amplitude')
    plt.ylabel('Probability')
    plt.title('Weight amplitude distribution')
    plt.grid(True)
    plt.show()
    
    
def print_data(data):
    ax = sns.heatmap(data, linewidth=0.5)
    plt.show()

And our function to convert tensors to integers with a precision parameter, and vice-versa

In [38]:
def fix_precision(tensor, precision_bits, rm_outlier_frac=100, parameter=True):
    tensor = (tensor * 2**precision_bits).long()
    max_value = max(
        np.abs(np.percentile(tensor, rm_outlier_frac)),
        np.abs(np.percentile(tensor, 100 - rm_outlier_frac))
    )
    cp_tensor = 1 * tensor
    tensor = tensor.clamp(min=-max_value, max=max_value)
    if parameter:
        return nn.Parameter(tensor, requires_grad=False)
    else:
        return tensor
    
def float_precision(tensor, precision_bits):
    tensor = tensor.float()/2**precision_bits
    return tensor

In [39]:
class Parser:
    """Parameters for the testing"""
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 10

In [40]:
args = Parser()

data = learn.load_data()
train_data, train_target_char, train_target_family, test_data, test_target_char, test_target_family = data
test_target = test_target_char
test_dataset = learn.build_tensor_dataset(test_data, test_target)
test_loader = utils.DataLoader(
    test_dataset,
    batch_size=args.test_batch_size, shuffle=True
)

Training set 60000 items
Testing set  10000 items


Our test function modifies the data first in the encrypted setting (ie values are integers) and then converts back the output to float to run the public part of the net used to predict digits.

In [41]:
def test(model, test_loader, prec):
    data_prec, proj_prec, diag_prec = prec 
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # Private Part
            data = fix_precision(data, data_prec, parameter=False)
            private_output = model.quad(data)
            # Public Part
            output = float_precision(private_output, sum(prec))
            output = model.char_net(output)

            pred = output.argmax(1, keepdim=True)  # get the index of the max log-probability

            correct += pred.eq(target.view_as(pred)).sum().item()

    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(test_loader.dataset), acc))
    
    return acc

Given a precision setting, this returns the accuracy of the main task and the maximum number of bits needed to store the output.

In [178]:
def test_compression(prec, rm_outlier_frac=100, show_distrib=False, cv=6):
    data = collateral.data.load_resistance_data(font1=None, font2=None, cv=cv)
    train_data, train_target_char, train_target_family, test_data, test_target_char, test_target_family = data
    # Merge the target datasets
    train_target = list(zip(train_target_char, train_target_family))
    test_target = list(zip(test_target_char, test_target_family))

    # We use here the slightly modified version of this function
    train_dataset = learn.utils.build_tensor_dataset(train_data, train_target_char)
    test_dataset = learn.utils.build_tensor_dataset(test_data, test_target_char)
    
    model = CollateralNet(private_output_size=PRIVATE_OUTPUT_SIZE)
    model.load_state_dict(torch.load(path))
    model.eval()

    data_prec, proj_prec, diag_prec = prec 
    
    if show_distrib and False:
        print_hist(model.proj1.weight.detach())
        print_hist(model.proj1.bias.detach())
        print_hist(model.diag1.weight.detach())
        
    # Convert the model
    model.proj1.weight = fix_precision(model.proj1.weight, proj_prec, rm_outlier_frac)
    model.proj1.bias = fix_precision(model.proj1.bias, proj_prec + data_prec, rm_outlier_frac)
    model.diag1.weight = fix_precision(model.diag1.weight, diag_prec, rm_outlier_frac)
        
    if show_distrib:
        print_hist(model.proj1.weight)
        print_hist(model.proj1.bias)
        print_hist(model.diag1.weight)
    
    data_sample = fix_precision(test_dataset[0][0], data_prec, parameter=False)
    n_bits = (
        ceil(log2(torch.max(data_sample))) * 2 +
        ceil(log2(max(
            torch.max(model.proj1.bias / 2 ** data_prec),
            torch.max(model.proj1.weight)
        ) * 2)) * 2 + 
        ceil(log2(torch.max(model.diag1.weight) * 2))   
    )
    print("approx size", n_bits)
        
    test_dataset_prec = torch.utils.data.TensorDataset(
        fix_precision(test_dataset.tensors[0], data_prec, parameter=False),
        test_dataset.tensors[1]
    )
    test_loader_prec = torch.utils.data.DataLoader(test_dataset_prec, batch_size=args.test_batch_size)
    acc_char = test(model, test_loader_prec, prec)
    
    return model, data_prec, acc_char
    

In [179]:
prec=(3, 7, 5) # (3, 7, 5)
data_prec, proj_prec, diag_prec = prec 
model, data_prec, acc_char = test_compression(prec=prec, rm_outlier_frac=99.9, show_distrib=False)

Training set 60000 items
Testing set  10000 items
approx size 19

Test set: Accuracy: 9953/10000 (99.53%)



In [181]:
acc_chars = [] 
for cv in range(7):
    path = f'models/quadconvnet_0.5_0.002_1.7_4_{cv}_par2.pt'
    prec=(3, 7, 5) # (3, 7, 5)
    data_prec, proj_prec, diag_prec = prec 
    model, data_prec, acc_char = test_compression(prec=prec, rm_outlier_frac=99.9, show_distrib=False, cv=cv)
    acc_chars.append(acc_char)

print(np.mean(acc_chars), np.std(acc_chars))

Training set 60000 items
Testing set  10000 items
approx size 17

Test set: Accuracy: 9782/10000 (97.82%)

Training set 60000 items
Testing set  10000 items
approx size 19

Test set: Accuracy: 9789/10000 (97.89%)

Training set 60000 items
Testing set  10000 items
approx size 19

Test set: Accuracy: 9705/10000 (97.05%)

Training set 60000 items
Testing set  10000 items
approx size 17

Test set: Accuracy: 9761/10000 (97.61%)

Training set 60000 items
Testing set  10000 items
approx size 19

Test set: Accuracy: 9775/10000 (97.75%)

Training set 60000 items
Testing set  10000 items
approx size 19

Test set: Accuracy: 9805/10000 (98.05%)

Training set 60000 items
Testing set  10000 items
approx size 19

Test set: Accuracy: 9789/10000 (97.89%)

97.72285714285714 0.3020271646115056


So, as you observe, we keep a 98% accuracy with weight and data inputs integers between 0 and 16 (so on 4 bits)!.
Experiments show that the total output is around 27 bits long, so we'll construct a discrete log algorithm which can work for output up to 30 bits long (so < 10^9)


Let's save this model now

In [44]:
saving_path = '/Users/tryffel/code/reading-in-the-dark/mnist/objects/ml_models/quad_conv.pt'
torch.save(model.state_dict(), saving_path)

In [62]:
proj = model.proj1.weight.t().tolist()
proj.append((model.proj1.bias / 8).tolist())
diag = model.diag1.weight.t().tolist()

In [63]:
len(proj), len(proj[0])

(785, 40)

In [64]:
len(diag), len(diag[0])

(40, 4)

In [67]:
print(proj)
print(diag)

[[3, -5, -3, -1, 0, 3, -3, 0, 2, 2, 0, 1, -2, 2, 3, 0, 1, -1, -1, 2, 0, 3, -2, 1, 1, 0, 0, 2, 1, -1, 0, 1, -2, 3, -3, -3, -1, 2, -4, -1], [0, -1, 0, 0, -3, 3, 1, 0, 1, 4, 3, -3, 4, 0, -3, -4, 1, -1, -1, -1, -1, 3, -2, 1, -3, 0, -2, 2, 0, -2, 1, 0, -3, -3, 0, 1, 2, 3, -2, 0], [1, -1, -1, -1, -2, -3, -3, -1, 1, -1, -1, -2, 4, -1, -3, 3, -1, -3, -2, -1, -1, 3, 0, -3, 0, 2, -1, 1, -4, -1, -1, -1, 1, 3, 2, 1, 2, -2, -2, -3], [3, -2, -3, 4, 0, 0, 1, 1, 2, 1, -3, 0, 3, 0, 0, 2, 3, 4, 3, -2, 0, 3, -4, -3, 3, -1, 0, -1, 2, 2, -2, -4, -4, 2, 1, -2, -3, 0, -3, -1], [0, -5, 1, 0, -2, 0, 3, -2, 0, -3, -2, -4, 0, 3, 3, -3, -3, -4, 0, -2, 1, 0, -4, -3, 0, 2, 2, -2, 1, 3, -3, 3, 0, -2, 0, 2, 1, 4, 1, -1], [2, 0, 0, -1, 4, 1, 0, 0, 0, 2, 4, -3, 4, 0, 2, 1, 4, 0, 1, 3, -2, 0, 0, 3, 3, 1, 0, 2, 3, -1, -2, 3, 3, 1, -3, 3, 4, -1, 0, 0], [1, -6, 0, -3, 2, 3, -3, -2, -2, 4, -3, -2, 1, -3, 1, -4, -3, 0, 2, 0, -2, -2, -1, 1, -1, -3, 0, 2, -2, 0, -1, 3, 0, 4, 4, -2, -1, 4, -3, 0], [1, -4, -1, 3, 2, 0, -3, 0, -2

## Collateral efficiency
Let's also test the impact of weight conversion and compression the the CNN model against which resistance had been built. You can observe that there is no visible impact.

In [114]:
import torch.optim as optim

In [132]:
class Parser:
    """Parameters for the training"""
    def __init__(self):
        self.epochs = 10
        self.sabotage_epochs = 50
        self.new_adversary_epochs = 50
        self.lr = 0.002001
        self.momentum = 0.5
        self.test_batch_size = 1000
        self.batch_size = 64
        self.log_interval = 100

In [143]:
def test_collateral(prec, rm_outlier_frac):
    model = CollateralNet(private_output_size=PRIVATE_OUTPUT_SIZE)
    model.load_state_dict(torch.load(path))
    model.eval()

    data_prec, proj_prec, diag_prec = prec 
        
    # Convert the model
    model.proj1.weight = nn.Parameter(
        fix_precision(model.proj1.weight, proj_prec, rm_outlier_frac).float() / 2**proj_prec
        , requires_grad=False)
    model.proj1.bias = nn.Parameter(
        fix_precision(model.proj1.bias, proj_prec + data_prec, rm_outlier_frac).float() / 2**(proj_prec + data_prec)
        , requires_grad=False)
    model.diag1.weight = nn.Parameter(
        fix_precision(model.diag1.weight, diag_prec, rm_outlier_frac).float() / 2**diag_prec
        , requires_grad=False)
        
    
    args = Parser()
    alpha = 1.7
    
    train_dataset, test_dataset = collateral.data.get_collateral_datasets(*fonts, letter)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    
    train_dataset_prec = torch.utils.data.TensorDataset(
        fix_precision(train_dataset.tensors[0], data_prec, parameter=False).float() / 2**data_prec,
        train_dataset.tensors[1]
    )
    train_loader_prec = torch.utils.data.DataLoader(train_dataset_prec, batch_size=args.batch_size)
    
    test_dataset_prec = torch.utils.data.TensorDataset(
        fix_precision(test_dataset.tensors[0], data_prec, parameter=False).float() / 2**data_prec,
        test_dataset.tensors[1]
    )
    test_loader_prec = torch.utils.data.DataLoader(test_dataset_prec, batch_size=args.test_batch_size)
    
    test_perfs_char = []
    test_perfs_font = []
    for epoch in range(1, args.new_adversary_epochs + 1):
        
        initial_phase = False
        perturbate = False
        recover = True
        new_adversary = False
        
        collateral.train(
            args, model, train_loader_prec, optimizer, epoch, alpha,
            initial_phase, perturbate, recover, new_adversary
        )
        test_perf_char, test_perf_font = collateral.test(args, model, test_loader_prec, new_adversary)
        test_perfs_char.append(test_perf_char)
        test_perfs_font.append(test_perf_font)

    return test_perfs_char, test_perfs_font
    

In [144]:
prec=(3, 7, 5)
test_perfs_char, test_perfs_font = test_collateral(prec, rm_outlier_frac=99.9)

Training set 60000 items
Testing set  10000 items
Train Epoch: 1 [0/60000 (0%)]	Loss Char: 0.000000 Loss Font: 0.696381
Train Epoch: 1 [6400/60000 (11%)]	Loss Char: 0.000000 Loss Font: 0.688837
Train Epoch: 1 [12800/60000 (21%)]	Loss Char: 0.000000 Loss Font: 0.694149
Train Epoch: 1 [19200/60000 (32%)]	Loss Char: 0.000000 Loss Font: 0.695527
Train Epoch: 1 [25600/60000 (43%)]	Loss Char: 0.000000 Loss Font: 0.681827
Train Epoch: 1 [32000/60000 (53%)]	Loss Char: 0.000000 Loss Font: 0.700897
Train Epoch: 1 [38400/60000 (64%)]	Loss Char: 0.000000 Loss Font: 0.693258
Train Epoch: 1 [44800/60000 (75%)]	Loss Char: 0.000000 Loss Font: 0.691493
Train Epoch: 1 [51200/60000 (85%)]	Loss Char: 0.000000 Loss Font: 0.688664
Train Epoch: 1 [57600/60000 (96%)]	Loss Char: 0.000000 Loss Font: 0.689833

Test set: Accuracy Char : 9988/10000 (99.88%)
          Accuracy Font : 5274/10000 (52.74%)
Train Epoch: 2 [0/60000 (0%)]	Loss Char: 0.000000 Loss Font: 0.681593
Train Epoch: 2 [6400/60000 (11%)]	Loss Char

Train Epoch: 10 [57600/60000 (96%)]	Loss Char: 0.000000 Loss Font: 0.687288

Test set: Accuracy Char : 9999/10000 (99.99%)
          Accuracy Font : 5369/10000 (53.69%)
Train Epoch: 11 [0/60000 (0%)]	Loss Char: 0.000000 Loss Font: 0.683382
Train Epoch: 11 [6400/60000 (11%)]	Loss Char: 0.000000 Loss Font: 0.679464
Train Epoch: 11 [12800/60000 (21%)]	Loss Char: 0.000000 Loss Font: 0.691261
Train Epoch: 11 [19200/60000 (32%)]	Loss Char: 0.000000 Loss Font: 0.692540
Train Epoch: 11 [25600/60000 (43%)]	Loss Char: 0.000000 Loss Font: 0.663808
Train Epoch: 11 [32000/60000 (53%)]	Loss Char: 0.000000 Loss Font: 0.666749
Train Epoch: 11 [38400/60000 (64%)]	Loss Char: 0.000000 Loss Font: 0.696775
Train Epoch: 11 [44800/60000 (75%)]	Loss Char: 0.000000 Loss Font: 0.681743
Train Epoch: 11 [51200/60000 (85%)]	Loss Char: 0.000000 Loss Font: 0.668213
Train Epoch: 11 [57600/60000 (96%)]	Loss Char: 0.000000 Loss Font: 0.685006

Test set: Accuracy Char : 9999/10000 (99.99%)
          Accuracy Font : 5386

Train Epoch: 20 [38400/60000 (64%)]	Loss Char: 0.000000 Loss Font: 0.696285
Train Epoch: 20 [44800/60000 (75%)]	Loss Char: 0.000000 Loss Font: 0.682676
Train Epoch: 20 [51200/60000 (85%)]	Loss Char: 0.000000 Loss Font: 0.664621
Train Epoch: 20 [57600/60000 (96%)]	Loss Char: 0.000000 Loss Font: 0.681711

Test set: Accuracy Char : 10000/10000 (100.00%)
          Accuracy Font : 5527/10000 (55.27%)
Train Epoch: 21 [0/60000 (0%)]	Loss Char: 0.000000 Loss Font: 0.683505
Train Epoch: 21 [6400/60000 (11%)]	Loss Char: 0.000000 Loss Font: 0.668380
Train Epoch: 21 [12800/60000 (21%)]	Loss Char: 0.000000 Loss Font: 0.683663


KeyboardInterrupt: 